# Using the NengoEdge Coral Runner

In this example we will walk through loading and running a model exported from NengoEdge
that's been uniquely configured to run on Coral's edge TPU dev board. The goal of this
demo is to provide a template for you to make your own custom TPU accelerated
applications using the NengoEdge runner.

## Installation

First, install NengoEdge tools using
[these instructions](https://www.nengo.ai/nengo-edge/developers.html).
Feel free to skip the Tensorflow step since we won't need it for this
runner.

For the Coral example, it's assumed that you have completed the Coral.ai
[getting started guide](https://coral.ai/docs/dev-board/get-started/) such that,

- The `tflite_runtime` Python package is installed on the board
- The Edge TPU delegates are installed (e.g.`libedgetpu.so.1` for Linux systems)
- You are able to ssh into your local board (e.g `ssh mendel@<CORAL_HOSTNAME>`; see
  [tutorial](https://coral.ai/docs/dev-board/mdt/#connect-using-other-ssh-tools) if not
  yet set up)

## Train a model in NengoEdge

The first step is to train a model in NengoEdge for the **Coral Dev Board** hardware.
See
[this blog
post](https://appliedbrainresearch.com/blog/fast-keyword-detection-with-lmus-on-gpu)
for a detailed walkthrough on how to train such a model.

## Export the trained model

When exporting the model from NengoEdge you must choose the "BINARY" option to get a
model targeted to run on the Edge TPU. The downloaded artifacts can be unpacked to a
directory of your choice, and for the purpose of this demo we'll assume the contents
have been unpacked to a directory called `coral_demo/`.

Inside this directory you'll find two files:

- `model_edgetpu.tflite`
- `parameters.json`

We'll create a `CoralRunner` that utilizes these artifacts.

Note that if you are running this code locally, you will need to uncomment the
`os.environ` line and update `<CORAL_HOSTNAME>` to point to the SSH hostname
(see the Installation steps above).

In [ ]:
import os
import numpy as np

from nengo_edge import CoralRunner

# os.environ["CORAL_HOSTNAME"] = <CORAL_HOSTNAME>
runner = CoralRunner(
    directory="coral_demo",
    username="mendel",
    hostname=os.environ["CORAL_HOSTNAME"],
)

You will likely have particular audio inputs you are interested in identifying with the
edge key word spotting model, but we'll showcase the general run steps assuming a random
signal.

In [ ]:
audio_inputs = np.random.uniform(
    -1, 1, (1, runner.preprocessing["sample_rate"])
).astype("float32")

# Keyword labels for model outputs
labels = [
    "<silence>",
    "<unknown>",
    "yes",
    "no",
    "up",
    "down",
    "left",
    "right",
    "on",
    "off",
    "stop",
    "go",
]

And finally we can run the model using the Coral device.

In [ ]:
outputs = runner.run(audio_inputs)
pred_label = np.argmax(outputs)

print(f"Predicted keyword: {labels[pred_label]}")

Since we used a random audio sample here and the model was trained on real audio
samples, it's likely this particular classification will
result in a random label. Note that the first `runner.run(inputs)` call will copy
over necessary runtime files that persist for the entire session. Subsequent calls to
run will only copy inputs and outputs to/from the Coral board. To reset this behaviour,
run `runner.reset()`. You'll notice that in this example we only ran the model on a
single batch of inputs but the runner also supports batched inputs.

That's it! You're now set to take this runner into your own applications.